[사용자 정의 데이터셋]
- pytorch에서 딥러닝 시 대량의 데이터 사용에 따른 부하, 많은 시간 소요에 대한 해결책으로 제시
- 대량 데이터셋 전용 처리 모듈 제공
- 종류
    - DataSet
        - 데이터셋 전처리, 텐서화 등의 작업 진행
    - DataLoader
        - DataSet 인스턴스 사용해 배치크기만큼 데이터 추출
        

[1] 모듈 로딩 및 데이터 준비

In [4]:
#모듈 로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
data_file='../data/iris.csv'

In [3]:
irisdf=pd.read_csv(data_file)
irisdf.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [5]:
#타겟 컬럼 수치화  w. LabeEncoder
encoder=LabelEncoder()
encoder.fit(irisdf['variety'])
irisdf['variety']=encoder.transform(irisdf['variety'])

[2] 사용자 정의 데이터셋 클래스 생성
- - -
- 클래스 목적: 학습용 데이터셋 텐서화 및 전처리
- 클래스 이름: custom_dataset
- 부모 클래스: torch.utils.data.Dataset
- 매개변수: featuredf,targetdf

In [6]:
class custom_dataset(Dataset):
    #데이터 로딩 및 전처리 진행, 인스턴스 생성 메서드
    def __init__(self,feature_df,target_df):

        #부모 클래스 생성
        super().__init__()
        self.feature_df=feature_df
        self.target_df=target_df
        self.n_rows=feature_df.shape[0]
        self.n_features=feature_df.shape[1]

    #데이터 개수 반환
    def __len__(self):
        return self.n_rows

    #특정 index의 데이터와 타겟 반환 메서드
    def __getitem__(self, idx):
        feature_ts=torch.FloatTensor(self.feature_df.iloc[idx].values)
        target_ts=torch.FloatTensor(self.target_df.iloc[idx].values)
        return feature_ts,target_ts

In [ ]:
#함수 기능: 파일 확장자별 데이터프레임 변환
#함수 이름: convertDF
#매개변수: data_path (파일 경로)
#함수 결과: DataFrame

def convertDF(data_path,exist_header=0):
    ext=data_path.rsplit('.')[-1]

    if ext=='csv':
        return pd.read_csv(data_path,header=exist_header)

    elif ext=='json':
        return pd.read_json(data_path,header=exist_header)
    
    elif ext in ['xlsx', 'xls']:
        return pd.read_excel(data_path,header=exist_header)
    
    else:
        return pd.read_table(data_path,header=exist_header)


In [ ]:
#클래스 기능: 파일 기반의 데이터셋
#클래스 이름: file_dataset
#매개변수: data_path (파일 경로)
#부모 클래스: utils.data.Dataset

class file_dataset(Dataset):
    #데이터 로딩 및 전처리 진행, 인스턴스 생성 메서드
    def __init__(self,data_path):

        #부모 클래스 생성
        super().__init__()

        #데이터 파일 -> DataFrame
        data_df=convertDF(data_path)

        #DataFrame -> feature/target 추출
        self.feature_df=data_df[data_df.columns[:-1]]
        self.target_df=data_df[data_df.columns[-1:]]

        
        self.n_features=data_df.shape[1]
        self.n_rows=self.feature_df.shape[0]

    #데이터 개수 반환
    def __len__(self):
        return self.n_rows

    #특정 index의 데이터와 타겟 반환 메서드
    def __getitem__(self, idx):
        feature_ts=torch.FloatTensor(self.feature_df.iloc[idx].values)
        target_ts=torch.FloatTensor(self.target_df.iloc[idx].values)
        return feature_ts,target_ts

[3] 데이터셋 인스턴스 생성

In [13]:
irisdf.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [19]:
#feature/target 추출
feature_df, target_df=irisdf[irisdf.columns[:-1]],irisdf[[irisdf.columns[-1]]]
                                                # = irisdf[irisdf.columns[-1:]]                          
print(f'feature_df: {feature_df.shape}')
print(f'target_df: {target_df.shape}')

#iris 데이터셋 인스턴스 생성
irisds=custom_dataset(feature_df, target_df)

feature_df: (150, 4)
target_df: (150, 1)


In [20]:
#IRIS 데이터셋 속성
irisds.feature_df

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [21]:
irisds.n_features, irisds.n_rows

(4, 150)

In [22]:
#iris 데이터셋 메서드
irisds[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] DataLoader 인스턴스 생성
- dataset 인스턴스, 배치 사이즈 필요

In [29]:
irisdl=DataLoader(irisds,batch_size=32)

In [31]:
for data_ts,target_ts in irisdl:
    print(data_ts.shape,target_ts.shape)
    break

torch.Size([32, 4]) torch.Size([32, 1])
